### Understanding the Dots and Boxes Game Implementation

#### Overview

**Dots and Boxes** is a two-player game where players take turns drawing lines between dots on a grid. The objective is to complete boxes by drawing the fourth side of the box. The player who completes a box gets an extra turn and scores a point.

#### Data Structures

1. **Grid Representation**:
    - The game board is represented as a 2D list of dictionaries.
    - Each dictionary holds the state of the horizontal (`'h'`) and vertical (`'v'`) lines connecting the dots.
    - The grid structure is created based on the number of rows and columns specified for the game. For example, a 5x5 grid will have 5 rows and 5 columns of dictionaries.

In [ ]:
self.grid = [[{'h': False, 'v': False} for _ in range(self.cols)] for _ in range(self.rows)]

  **Here**: 
  - `self.grid` is initialized to a 2D list where each element is a dictionary with two keys: `'h'` for horizontal lines and `'v'` for vertical lines. Both keys are initially set to `False`, indicating that no lines have been drawn yet.

2. **Box Tracking**:
    - The game also needs to keep track of which player completes each box. This is done using another 2D list called `self.boxes`.
    - Each element in `self.boxes` represents a box on the grid, and it holds `None` if the box is not yet completed, or the player number (1 or 2) who completed the box.
    - The size of `self.boxes` is one less than the grid in both dimensions because a box is formed between four dots.

In [ ]:
self.boxes = [[None for _ in range(self.cols-1)] for _ in range(self.rows-1)]

  **.**
  - `self.boxes` is initialized to a 2D list where each element is `None`, indicating that no boxes have been completed at the start of the game.

3. **Scores and Current Player**:
    - The game keeps track of the scores of both players using a dictionary `self.scores`.
    - `self.current_player` is a variable that keeps track of whose turn it is to play.

In [ ]:
self.scores = {1: 0, 2: 0}
self.current_player = 1

  **.** 
  - `self.scores` is a dictionary with keys 1 and 2, representing the two players. The values are their respective scores, which start at 0.
  - `self.current_player` is set to 1 at the start, indicating that Player 1 will make the first move.

#### Key Functions

1. **Initialization and Reset**:
    - `__init__(self, rows, cols)`: Initializes the game board with the specified rows and columns.
    - The constructor ensures that the grid size is at least 2x2 to form valid boxes.

In [ ]:
def __init__(self, rows, cols):
    if rows < 2 or cols < 2:
        raise ValueError("Grid size must be at least 2x2")
    self.rows = rows
    self.cols = cols
    self.reset()

  **.** 
  - `reset(self)`: Resets the game to its initial state, clearing the grid and resetting scores.
  - The grid and boxes are reset to their initial states, and the current player is set to 1.

In [ ]:
def reset(self):
    self.grid = [[{'h': False, 'v': False} for _ in range(self.cols)] for _ in range(self.rows)]
    self.boxes = [[None for _ in range(self.cols-1)] for _ in range(self.rows-1)]
    self.current_player = 1
    self.scores = {1: 0, 2: 0}
    print("Game reset")

2. **Drawing Lines**:
    - `draw_line(self, start, end)`: Main method to draw a line between two points if the move is valid. Updates the grid, checks for box completion, and manages player turns.
    - When a player makes a move, they specify the start and end points of the line they wish to draw. This move is first validated to ensure it is within the bounds of the grid, forms a straight line, and has not already been drawn. If the move is valid, the grid is updated to reflect the new line by setting the corresponding `'h'` or `'v'` value to `True` in the `self.grid`. The game then checks if the new line completes any boxes by verifying if all four sides of a box are drawn. If a box is completed, it updates the `self.boxes` and the scores, and the current player gets another turn; otherwise, the turn switches to the other player. If the game is over (i.e., all boxes are completed), the winner is determined based on the scores.

In [ ]:
def draw_line(self, start, end):
    try:
        if start is None or end is None:
            print("Error: Missing required positional arguments 'start' and 'end'")
            return False

        if not self.is_valid_input(start, end):
            print(f"Invalid input format: start={start}, end={end}")
            return False

        if self.is_game_over():
            self.get_winner()
            return False

        print(f"Player {self.current_player} attempting to draw line from {start} to {end}")
        if self.is_valid_move(start, end):
            print(f"Move from {start} to {end} is valid")
            self.update_grid(start, end)
            completed_box = self.check_and_update_boxes(start, end)
            self.display_grid_and_boxes()
            if completed_box:
                print(f"A box was completed by player {self.current_player}. The current player gets another turn.")
                return True  # Player gets another turn
            else:
                self.current_player = 3 - self.current_player
                print(f"No box completed. Switching to player {self.current_player}")
                return False
        else:
            self.print_invalid_move_message(start, end)
            return False

    except Exception as e:
        logging.error(f"Error: {e}. Input start: {start}, end: {end}")
        print(f"An unexpected error occurred. Please check the log file for details.")
        return False

  **.** 
  - `is_valid_move(self, start, end)`: Checks if the move is within bounds, a straight line, and not already drawn.
  - This method combines several checks to ensure that the move is legal in the context of the game.

In [ ]:
def is_valid_move(self, start, end):
    return (self.is_within_bounds(start, end) and
            self.is_straight_line(start, end) and
            not self.is_line_already_drawn(start, end))

  **.** 
  - `is_within_bounds(self, start, end)`: Ensures the move is within the grid boundaries.
  - This method checks if both the starting and ending points of the move are within the bounds of the grid.

In [ ]:
def is_within_bounds(self, start, end):
    return (0 <= start[0] < self.rows and 0 <= start[1] < self.cols and
            0 <= end[0] < self.rows and 0 <= end[1] < self.cols)

  **.**
  - `is_straight_line(self, start, end)`: Ensures the move is a straight line between adjacent dots.
  - This method checks if the move is either horizontal or vertical and covers exactly one unit distance.

In [ ]:
def is_straight_line(self, start, end):
    return (start[0] == end[0] and abs(start[1] - end[1]) == 1) or (start[1] == end[1] and abs(start[0] - end[0]) == 1)

  **.** 
  - `is_line_already_drawn(self, start, end)`: Checks if the line between the two points is already drawn.
  - This method verifies that the specified line hasn't already been drawn by checking the grid's state.

In [ ]:
def is_line_already_drawn(self, start, end):
    if start[0] == end[0]:
        return self.grid[start[0]][min(start[1], end[1])]['h']
    else:
        return self.grid[min(start[0], end[0])][start[1]]['v']

3. **Grid Updates and Box Completion**:
    - `update_grid(self, start, end)`: Updates the grid to reflect the newly drawn line.
    - This method updates the state of the grid to mark the newly drawn line as occupied.

In [ ]:
def update_grid(self, start, end):
    if start[0] == end[0]:
        self.grid[start[0]][min(start[1], end[1])]['h'] = True
        print(f"Horizontal line drawn at row {start[0]} between columns {start[1]} and {end[1]}")
    else:
        self.grid[min(start[0], end[0])][start[1]]['v'] = True
        print(f"Vertical line drawn at column {start[1]} between rows {start[0]} and {end[0]}")

  **.** 
  - `check_and_update_boxes(self, start, end)`: Checks if any boxes are completed by the new line and updates scores and player turns accordingly.
  - This method checks the surrounding boxes of the newly drawn line to see if any box is completed, updates the scores, and keeps track of box ownership.

In [ ]:
def check_and_update_boxes(self, start, end):
    completed_box = False
    for box in self.get_surrounding_boxes(start, end):
        print(f"Checking potential box completion at {box}")
        if self.is_box_complete(box):
            print(f"Box completed at {box} by player {self.current_player}")
            self.boxes[box[0]][box[1]] = self.current_player
            self.scores[self.current_player] += 1
            completed_box = True
    return completed_box

  **.** 
  - `get_surrounding_boxes(self, start, end)`: Identifies the boxes that could be affected by the newly drawn line.
  - This method determines which boxes might be completed based on the newly drawn line by checking the adjacent positions.

In [ ]:
def get_surrounding_boxes(self, start, end):
    boxes = []
    if start[0] > 0 and end[0] > 0 and start[0] == end[0]:  # Horizontal line
        boxes.append((start[0]-1, min(start[1], end[1])))
    if start[0] < self.rows - 1 and end[0] < self.rows - 1 and start[0] == end[0]:  # Horizontal line
        boxes.append((start[0], min(start[1], end[1])))
    if start[1] > 0 and end[1] > 0 and start[1] == end[1]:  # Vertical line
        boxes.append((min(start[0], end[0]), start[1]-1))
    if start[1] < self.cols - 1 and end[1] < self.cols - 1 and start[1] == end[1]:  # Vertical line
        boxes.append((min(start[0], end[0]), start[1]))
    return boxes

  **.** 
  - `is_box_complete(self, box)`: Checks if a specified box is complete by checking the state of its four sides.
  - This method verifies if all four sides of the specified box have been drawn.

In [ ]:
def is_box_complete(self, box):
    r, c = box
    complete = (
        self.grid[r][c]['h'] and self.grid[r+1][c]['h'] and
        self.grid[r][c]['v'] and self.grid[r][c+1]['v']
    )
    print(f"Checking box at {box}: horizontal top {self.grid[r][c]['h']}, horizontal bottom {self.grid[r+1][c]['h']}, vertical left {self.grid[r][c]['v']}, vertical right {self.grid[r][c+1]['v']}")
    if complete:
        print(f"Box at {box} is complete.")
    return complete

4. **Game State Checks**:
    - `is_game_over(self)`: Checks if the game is over by verifying if all boxes are claimed.
    - This method iterates through the boxes to check if all of them have been completed.

In [ ]:
def is_game_over(self):
    for row in self.boxes:
        for box in row:
            if box is None:
                return False
    return True

  **.** 
  - `get_winner(self)`: Determines the winner based on scores and handles ties gracefully.
  - This method compares the scores of the two players and announces the winner, or declares a tie if the scores are equal.

In [ ]:
def get_winner(self):
    print(f"Final Scores: Player 1: {self.scores[1]}, Player 2: {self.scores[2]}")
    if self.scores[1] > self.scores[2]:
        print("Player 1 wins!")
        return 1
    elif self.scores[2] > self.scores[1]:
        print("Player 2 wins!")
        return 2
    else:
        print("It's a tie!")
        return None

5. **Display Functions**:
    - `display(self)`: Displays the current state of the game board and scores.
    - This method prints the current state of the grid, showing which lines have been drawn and the scores of both players.

In [ ]:
def display(self):
    for r in range(self.rows):
        line = ""
        for c in range(self.cols):
            line += "o"
            if c < self.cols - 1:
                line += "---" if self.grid[r][c]['h'] else "   "
        print(line)
        if r < self.rows - 1:
            line = ""
            for c in range(self.cols):
                line += "|" if self.grid[r][c]['v'] else " "
                if c < self.cols - 1:
                    line += "   "
            print(line)
    print("Scores:", self.scores)
    print(f"Player {self.current_player}'s turn.")

  **.** 
  - `display_grid_and_boxes(self)`: Displays the grid and boxes for debugging purposes.
  - This method prints the current state of the grid and the boxes, useful for debugging.

In [ ]:
def display_grid_and_boxes(self):
    print("\nGrid state:")
    for r in range(self.rows):
        line = ""
        for c in range(self.cols):
            line += "o"
            if c < self.cols - 1:
                line += "---" if self.grid[r][c]['h'] else "   "
        print(line)
        if r < self.rows - 1:
            line = ""
            for c in range(self.cols):
                line += "|" if self.grid[r][c]['v'] else " "
                if c < self.cols - 1:
                    line += "   "
            print(line)

    print("\nBoxes state:")
    for r in range(len(self.boxes)):
        print(self.boxes[r])
    print("\n")

### Additional Considerations

1. **Edge Case Handling**:
    - **Boundary Moves**: Ensure moves at the edges of the grid are correctly handled.
    - **Negative Indices**: Ensure negative indices are checked and handled.
    - This method checks if both the starting and ending points of the move are within the bounds of the grid.

In [ ]:
def is_within_bounds(self, start, end):
    return (0 <= start[0] < self.rows and 0 <= start[1] < self.cols and
            0 <= end[0] < self.rows and 0 <= end[1] < self.cols)

  **.** 
  - **Minimum Grid Size**: Validate that the grid size is at least 2x2 during initialization.
  - This ensures that the grid is large enough to form at least one box.

In [ ]:
def __init__(self, rows, cols):
    if rows < 2 or cols < 2:
        raise ValueError("Grid size must be at least 2x2")
    self.rows = rows
    self.cols = cols
    self.reset()

2. **Player Turn Management**:
    - Ensure the current player is correctly switched after a valid move that doesn't complete a box.
    - If a box is completed, the same player gets another turn; otherwise, the turn switches to the other player.

In [ ]:
def draw_line(self, start, end):
    if self.is_valid_move(start, end):
        self.update_grid(start, end)
        completed_box = self.check_and_update_boxes(start, end)
        if completed_box:
            return True  # Player gets another turn
        else:
            self.current_player = 3 - self.current_player
            return False

    - Handle consecutive turns correctly when a player completes a box.
    - This ensures that the player who completes a box gets an extra turn.

In [ ]:
if completed_box:
    return True  # Player gets another turn

3. **Input Validation**:
    - Handle non-integer inputs gracefully.
    - This method checks if the inputs are tuples of integers and are of length 2.

In [ ]:
def is_valid_input(self, start, end):
    return (isinstance(start, tuple) and isinstance(end, tuple) and
            len(start) == 2 and len(end) == 2 and
            all(isinstance(i, int) for i in start + end))

    - Handle missing inputs without causing crashes.
    - This method ensures that both start and end points are provided.

In [ ]:
def draw_line(self, start=None, end=None):
    if start is None or end is None:
        print("Error: Missing required positional arguments 'start' and 'end'")
        return False

4. **Error Logging**:
    - Implement logging to capture unexpected errors for future debugging.
    - Log invalid input attempts for future review and handling.

In [ ]:
import logging
logging.basicConfig(filename='dots_and_boxes.log', level=logging.ERROR, format='%(asctime)s %(message)s')

try:
    # Code that might raise an exception
except Exception as e:
    logging.error(f"Error: {e}. Input start: {start}, end: {end}")
    print(f"An unexpected error occurred. Please check the log file for details.")
    return False

5. **User Feedback and Interaction**:
    - Provide clear prompts and feedback for player inputs during gameplay.
    - Give detailed feedback for invalid moves, specifying the reason.

In [ ]:
def print_invalid_move_message(self, start, end):
    if not self.is_within_bounds(start, end):
        print(f"Invalid move by player {self.current_player}: Move is out of bounds. Try again.")
    elif not self.is_straight_line(start, end):
        print(f"Invalid move by player {self.current_player}: Move is not a straight line. Try again.")
    elif self.is_line_already_drawn(start, end):
        print(f"Invalid move by player {self.current_player}: Line is already drawn. Try again.")
    else:
        print(f"Invalid move by player {self.current_player}: Unknown reason. Try again.")

### Example Workflow

1. **Initialization**:
    - Initialize the game with a specified grid size, ensuring the size is valid.

In [ ]:
game = DotsAndBoxes(5, 5)

2. **Drawing Lines**:
    - Players take turns to draw lines between dots.
    - Each move is validated for bounds, straightness, and whether the line is already drawn.

In [ ]:
game.draw_line((0, 0), (0, 1))  # Player 1
game.draw_line((0, 1), (1, 1))  # Player 2

3. **Checking for Box Completion**:
    - After each move, check if any boxes are completed.
    - Update the scores and manage turns accordingly.

In [ ]:
game.check_and_update_boxes((0, 0), (0, 1))

4. **Game Over and Winner Determination**:
    - The game ends when all boxes are completed.
    - Determine the winner based on scores, handling ties gracefully.

In [ ]:
if game.is_game_over():
    game.get_winner()

5. **Displaying the Game State**:
    - Regularly display the current state of the game board, scores, and whose turn it is.
    - Provide debugging information as needed.

In [ ]:
game.display()

### Example Scenario

1. **Player 1 draws a line** from (0, 0) to (0, 1).

In [ ]:
game.draw_line((0, 0), (0, 1))

2. **Player 2 draws a line** from (0, 1) to (1, 1).

In [ ]:
game.draw_line((0, 1), (1, 1))

3. **Player 1 draws a line** from (1, 1) to (1, 0).

In [ ]:
game.draw_line((1, 1), (1, 0))

4. **Player 1 draws a line** from (1, 0) to (0, 0), completing a box and earning an extra turn.

In [ ]:
game.draw_line((1, 0), (0, 0))  # Completes a box

5. **Game state is displayed** showing the updated grid, scores, and whose turn it is.

In [ ]:
game.display()